In [16]:
from sentence_transformers import SentenceTransformer
from docuverse.utils import open_stream, read_config_file
import pickle
from tqdm import tqdm, trange
from docuverse.utils.text_tiler import TextTiler
from docuverse.engines.search_data import SearchData
from docuverse.engines.data_template import DataTemplate
import os

In [50]:
model_name = "thenlper/gte-small"
model = SentenceTransformer(model_name)
max_size = 512
stride = 100
tiler = TextTiler(max_doc_size=max_size, stride=stride, tokenizer=model.tokenizer, aligned_on_sentences=True)

/home/raduf/miniforge3/envs/docuverse/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:

datadir="data/clapnq_small"

In [32]:
import pandas as pd
data = pd.read_csv(os.path.join(datadir, "passages.tsv"), sep="\t")
sents = data['text']

In [48]:
tiled_data = [tiler.create_tiles(i, txt, '', max_doc_size=50, stride=10) for i, txt in enumerate(sents[:10])]

In [ ]:
tiled_data

# Run the tiler in parallel

In [51]:
from docuverse.utils import parallel_process

def proc_text(item):
     res = tiler.create_tiles(item['id'], item['text'], item['title'])
     item['result'] = res
     return item

res = parallel_process(proc_text, [{'id': i, 'text': txt, 'title': f"Title for item {i}"} for i, txt in enumerate(sents[:100])], num_threads=10)

Processing result:::   0%|          | 0/100 [00:00<?, ?it/s]

In [52]:
res

[{'id': 0,
  'text': 'Egocentrism is the inability to differentiate between self and other . More specifically , it is the inability to untangle subjective schemas from objective reality ; an inability to understand or assume any perspective other than their own .',
  'title': 'Title for item 0',
  'result': [{'id': '0-0-242',
    'text': 'Title for item 0\nEgocentrism is the inability to differentiate between self and other . More specifically , it is the inability to untangle subjective schemas from objective reality ; an inability to understand or assume any perspective other than their own .'}]},
 {'id': 1,
  'text': "Although egocentrism and narcissism appear similar , they are not the same . A person who is egocentric believes they are the center of attention , like a narcissist , but does not receive gratification by one 's own admiration . Both egotists and narcissists are people whose egos are greatly influenced by the approval of others , while for egocentrists this may or ma